In [1]:
import warnings
warnings.filterwarnings('ignore')
import flowermd
import hoomd
import gsd
import matplotlib.pyplot as plt
import numpy as np
import gsd.hoomd
from flowermd.base import Pack, Simulation
from flowermd.library import OPLS_AA_PPS,PPS
from flowermd.utils import get_target_box_mass_density
import unyt as u
import hoomd

In [2]:
import signac
from flow import FlowProject, directives
import pickle

def get_ff(project,jobid):
    """"""
    msibi_project = signac.get_project(project)
    msibi_job = msibi_project.open_job(id=jobid)
    with open(msibi_job.fn("pps-msibi.pickle"), "rb") as f:
        hoomd_ff = pickle.load(f)
        
    for force in hoomd_ff:
        if isinstance(force, hoomd.md.bond.Table):
            print("Replacing bond table potential with harmonic")
            hoomd_ff.remove(force)
            harmonic_bond = hoomd.md.bond.Harmonic()
            harmonic_bond.params["A-A"] = dict(k=1777.6, r0=1.4226)
            hoomd_ff.append(harmonic_bond)
        else:
            pass
    return hoomd_ff

In [3]:
PPS_smiles = "c1ccc(S)cc1"
pps_chains = PPS(lengths=[15,17,22], num_mols=[3,2,2])
pps_chains.coarse_grain(beads={"A": PPS_smiles})
system = Pack(molecules=[pps_chains], density=0.1)
ff_file = "/home/erjank_project/pps-entanglement-length/Entanglements/angle-flow-with-pairs"
jobid = "34c9e9f8fa7d942743adbf6835395671"
ff = get_ff(ff_file,jobid)

Replacing bond table potential with harmonic


In [4]:
gsd_path=('cg-pps-trajectory0.gsd')
log_path=('cg-pps-log0.txt')
sim = Simulation.from_system(
    system=system,
    forcefield=ff,
    gsd_write_freq=int(1e5),
    gsd_file_name=gsd_path,
    log_file_name=log_path,
    dt=0.001,
    log_write_freq=int(1e5),
)
#target_box = get_target_box_mass_density(density=system.density, mass=system.mass.to("g"))
#lj_sim.run_update_volume(final_box_lengths=target_box,kT=5.0,n_steps=1e4,tau_kt=5*lj_sim.dt,period=10)
sim.run_NVT(n_steps=int(5e7), kT=5.0, tau_kt=10*sim.dt)
sim.flush_writers()
sim.save_restart_gsd("pps_restart.gsd")

Initializing simulation state from a gsd.hoomd.Frame.
Step 100000 of 50000000; TPS: 30784.11; ETA: 27.0 minutes
Step 200000 of 50000000; TPS: 32815.13; ETA: 25.3 minutes
Step 300000 of 50000000; TPS: 33392.35; ETA: 24.8 minutes
Step 400000 of 50000000; TPS: 33651.05; ETA: 24.6 minutes
Step 500000 of 50000000; TPS: 33807.22; ETA: 24.4 minutes
Step 600000 of 50000000; TPS: 33836.51; ETA: 24.3 minutes
Step 700000 of 50000000; TPS: 33867.65; ETA: 24.3 minutes
Step 800000 of 50000000; TPS: 33925.77; ETA: 24.2 minutes
Step 900000 of 50000000; TPS: 33945.79; ETA: 24.1 minutes
Step 1000000 of 50000000; TPS: 34007.75; ETA: 24.0 minutes
Step 1100000 of 50000000; TPS: 33972.23; ETA: 24.0 minutes
Step 1200000 of 50000000; TPS: 33942.33; ETA: 24.0 minutes
Step 1300000 of 50000000; TPS: 34020.46; ETA: 23.9 minutes
Step 1400000 of 50000000; TPS: 34000.63; ETA: 23.8 minutes
Step 1500000 of 50000000; TPS: 34031.19; ETA: 23.8 minutes
Step 1600000 of 50000000; TPS: 34087.6; ETA: 23.7 minutes
Step 1700000

In [5]:
from cmeutils.sampling import is_equilibrated
shrink_cut = 10
log = np.genfromtxt(log_path, names=True)
pe = log["mdcomputeThermodynamicQuantitiespotential_energy"]
# equil: bool (True or False), t0: int (starting data point that you can sample from), neff: number of data points in equilibrated region
print(pe[shrink_cut:])
equil, t0, g, neff = is_equilibrated(data=pe[shrink_cut:], threshold_neff=50) 
if equil:
    print("True")
else:
    # If it's not equilibrated, job.doc.equilibrated stays false, and the "run-longer" operation in project.py will run next time we submit this job
    print("False")

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************



[2205.76 2236.21 1936.77 2105.63 2327.76 2356.62 2056.85 2242.41 2443.92
 2234.76 2200.47 2284.32 1887.01 2316.79 2141.85 2166.86 2159.13 2285.97
 1950.38 2011.93 1968.8  2187.14 2221.41 1891.39 2074.67 2198.46 2234.4
 2117.75 2083.03 2145.21 2161.33 1927.93 2343.   2356.54 1920.05 2259.5
 2016.98 2530.23 2179.96 2266.73 2455.88 2422.12 2241.89 2118.69 2210.2
 1903.3  2104.71 2256.55 2200.56 2149.76 2563.89 2256.21 2345.41 1996.02
 2202.27 2279.67 2501.75 2210.45 2241.47 2136.86 2096.87 2234.89 1868.45
 2558.77 2055.65 1958.02 2466.58 2024.68 2182.91 1967.65 2161.31 1995.93
 2158.77 1899.24 2056.84 1944.48 2008.95 2185.73 2074.97 2207.11 2667.74
 2088.35 2242.16 2102.82 2255.63 1722.02 1964.61 2090.43 1911.69 2103.6
 2155.75 2010.39 2267.   2498.49 2226.5  1934.52 2121.71 2141.68 1978.09
 2372.88 2319.35 2502.08 2022.34 2291.62 2124.62 2313.39 2020.17 2132.78
 1920.13 2177.9  2434.89 2314.03 1840.7  2066.75 1877.7  1994.42 2130.99
 2268.41 2202.59 2313.1  2290.9  2254.62 2069.84 2152.2